# Anomalies

ANSI described some isolation levels that have to be implemented to deal with concurrency.

They are based on the different *Data Anomalies* that they allow in terms of concurrent transactions.

## Prepare Env

In [ ]:
DROP TABLE IF EXISTS users;
CREATE TABLE users(id INT, name VARCHAR(250), age INT);

INSERT INTO users VALUES
(1, 'Alice', 20),
(2, 'Bob', 25);
COMMIT;

## Dirty Read

**See uncommitted changes**

Occurs when a transaction retrieves a row that has been updated by another transaction that is not yet committed.

```SQL
/** Transaction 1 **/
BEGIN;
SELECT age FROM users WHERE id = 1;
-- Retrieves 20
```

```SQL
/** Transaction 2 **/
BEGIN;
UPDATE users SET age = 21 WHERE id = 1;
-- No commit!
```

```SQL
/** Transaction 1 **/
SELECT age FROM users WHERE id = 1;
-- Retrieves 21 -> CHANGED
-- In a serial database that would be still 20!
COMMIT;
```

## Non-Repeatable Read

**See committed changes on row between two queries**

Occurs when a transaction retrieves a row twice and that row is updated by another transaction that is committed in between

```SQL
/** Transaction 1 **/
BEGIN;
SELECT age FROM users WHERE id = 1;
-- Retrieves 20
```

```SQL
/** Transaction 2 **/
BEGIN;
UPDATE users SET age = 21 WHERE id = 1;
COMMIT; -- Committed!
```

```SQL
/** Transaction 1 **/
SELECT age FROM users WHERE id = 1;
-- Retrieves 21 -> CHANGED
-- In a serial database that would be still 20!
COMMIT;
```

## Phantom Read

**See committed changes on result set between two queries**

Occurs when a transaction retrieves a set of rows twice and new rows are inserted into or removed from that set by another transaction that is committed in between

```SQL
/** Transaction 1 **/
BEGIN;
SELECT name FROM users WHERE age > 17;
-- Retrieves [Alice, Bob]
```

```SQL
/** Transaction 2 **/
BEGIN;
INSERT INTO users VALUES (3, 'Carol', 26);
COMMIT; -- Committed!
```

```SQL
/** Transaction 1 **/
SELECT name FROM users WHERE age > 17;
-- Retrieves [Alice, Bob, Carol] -> CHANGED
-- In a serial database that would be still [Alice, Bob]!
COMMIT;
```

## Lost Updates

**Two concurrent updates overwrite each other**

A lost update is a scenario in which two or more transactions update the same row, but neither transaction is aware of the modification made by the other transaction, and the second change overwrites the first modification.

### Classic Scenario

This won't happen in Postgres

```SQL
/** Transaction 1 **/
BEGIN;
UPDATE users SET age = age + 5 WHERE name = 'Carol';
-- 26 + 5 -> 31
```

```SQL
/** Transaction 2 **/
BEGIN;
UPDATE users SET age = age + 10 WHERE name = 'Carol';
-- 26 + 10 -> 36
COMMIT; -- Commits to 36
```

```SQL
/** Transaction 1 **/
COMMIT; -- Commits to 31 -> Transaction 2 is overwritten
-- In a serial database this would end up with 26 + 5 + 10 = 41! 
```

### Possible Problematic Scenario

```SQL
UPDATE users SET age = 26 WHERE name = 'Carol';
COMMIT;
```

```SQL
/** Transaction 1 **/
BEGIN;
SELECT age INTO carols_age FROM users WHERE name = 'Carol';
SELECT * FROM carols_age;
```

```SQL
/** Transaction 2 **/
BEGIN;
UPDATE users SET age = age + 10 WHERE name = 'Carol';
SELECT age FROM users WHERE name = 'Carol';
-- 26 + 10 -> 36
COMMIT; -- Commits to 36
```

```SQL
/** Transaction 1 **/
UPDATE users SET age = (SELECT age FROM carols_age) + 5 WHERE name = 'Carol';
SELECT age FROM users WHERE name = 'Carol';
-- 26 + 5 -> 31
COMMIT; -- Commits to 31 -> Transaction 2 is overwritten
-- In a serial database this would end up with 26 + 5 + 10 = 41! 
```

Now try the same thing when transaction 1 is in `Repeatable Read` isolation level!